In [1]:
import pandas as pd

jobs_df = pd.read_csv('data/LinkedInJobs_MLDataset.csv')
jobs_df.head()

,Co_Nm,Co_Pg_Lstd,Emp_Cnt,Flw_Cnt,Job_Ttl,Job_Desc,Is_Supvsr,max_sal,med_sal,min_sal,...,st_code,is_remote,views,app_typ,app_is_off,xp_lvl,domain,has_post_domain,is_sponsored,base_comp
0,HearingLife,True,1171,11417,Hearing Care Provider,Overview\n\nHearingLife is a national hearing ...,False,5250.00,5250.00,5250.00,...,SC,0,9,OffsiteApply,True,Entry level,careers-demant.icims.com,True,0,1
1,"Metalcraft of Mayville, Inc.",True,300,2923,Shipping & Receiving Associate 2nd shift (Beav...,Metalcraft of Mayville\nMetalcraft of Mayville...,False,0.00,0.00,0.00,...,WI,0,0,OffsiteApply,True,Not Listed,www.click2apply.net,True,0,0
2,"U.S. Tsubaki Power Transmission, LLC",True,314,8487,"Manager, Engineering",The TSUBAKI name is synonymous with excellence...,True,0.00,0.00,0.00,...,AL,0,0,OffsiteApply,True,Not Listed,www.click2apply.net,True,0,0
3,Episcopal Communities & Services,True,36,305,Cook,descriptionTitle\n\n Looking for a great oppor...,False,22.27,22.27,22.27,...,CA,0,1,OffsiteApply,True,Entry level,jobs.apploi.com,True,0,1
4,"iHerb, LLC",True,1227,51933,Principal Cloud Security Architect (Remote),"Job Summary\nAt iHerb, we are on a mission to ...",False,275834.00,240895.00,205956.00,...,XF,1,0,OffsiteApply,True,Mid-Senior level,careers.iherb.com,True,0,1


In [2]:
import spacy
import re
from tqdm import tqdm

nlp = spacy.load('en_core_web_sm')
stop_words = list("\n firm type diploma closing date send level space job offer company firms oct connect jobs \
                  fcfa price spaces articles tax year cv job interview motivation letter advice questions mail \
                  work answers recruiters expired internship company send do activity sector have profile files \
                  company years address direction cv location".split())
jobs_df['Job_Desc'] = jobs_df['Job_Desc'].astype('str')
descriptions = [re.sub(r'[\d?,.;:!/-]', '', x).strip().lower() for x in jobs_df['Job_Desc'].values]

for i, doc in enumerate(tqdm(nlp.pipe(descriptions, n_process=12), total=len(descriptions))):
    descriptions[i] = " ".join([word.text for word in doc if word.pos_ in ['NOUN', 'VERB'] and word.text not in stop_words])

100%|██████████| 33246/33246 [04:28<00:00, 123.82it/s]


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_df=0.50, min_df=3)
doc_term_matrix = vectorizer.fit_transform(descriptions)

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV

search_params = {
  'n_components': [7, 8, 9, 10],
  'learning_decay': [.5]
}

model = LatentDirichletAllocation(learning_method='online')

gridsearch = GridSearchCV(model, param_grid=search_params, n_jobs=-1, verbose=10)
gridsearch.fit(doc_term_matrix)

print("Best Model's Params: ", gridsearch.best_params_)
print("Best Log Likelihood Score: ", gridsearch.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3

In [19]:
import random
from sklearn.decomposition import LatentDirichletAllocation
# import pyLDAvis
# import pyLDAvis.sklearn

topics_n = 10
LDAmodel = LatentDirichletAllocation(n_components=topics_n, max_iter=10, learning_method='online', learning_decay=0.5, verbose=2, n_jobs=-1)
LDAmodel.fit(doc_term_matrix)

[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.8s remaining:    5.7s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    1.0s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parall

iteration: 1 of max_iter: 10


[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Pa

iteration: 2 of max_iter: 10


[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_j

...

iteration: 9 of max_iter: 10


[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    0.1s finished
[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.0s remaining:    0.2s
[Pa

iteration: 10 of max_iter: 10


[Parallel(n_jobs=16)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    1.1s remaining:    8.0s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    1.2s finished


LatentDirichletAllocation(learning_decay=0.5, learning_method='online',
                          n_jobs=-1, verbose=2)

In [20]:
words = vectorizer.get_feature_names_out()

top_n = 10

for idx, topic in enumerate(tqdm(LDAmodel.components_)):
        print("Top {} words in topic #{}:".format(top_n, idx))
        print(", ".join([words[i]
                        for i in topic.argsort()[-top_n:]][::-1]))

100%|██████████| 10/10 [00:00<00:00, 1059.01it/s]

Top 10 words in topic #0:
benefits, people, paid, employees, career, time, help, role, make, life
Top 10 words in topic #1:
data, design, engineering, systems, software, development, security, system, knowledge, solutions
Top 10 words in topic #2:
sales, customer, customers, store, service, business, product, products, manager, relationships
Top 10 words in topic #3:
business, project, projects, client, development, ensure, teams, role, accounting, process
Top 10 words in topic #4:
equipment, safety, perform, maintenance, duties, quality, production, environment, manufacturing, food
Top 10 words in topic #5:
time, pets, career, leave, petco, want, paid, help, week, pay
Top 10 words in topic #6:
care, health, patients, healthcare, services, nurse, patient, nursing, hospital, registered
Top 10 words in topic #7:
status, employment, gender, disability, applicants, protected, race, information, orientation, color
Top 10 words in topic #8:
marketing, media, content, communication, community